# Préparation des données

## Inspection

In [19]:
import pandas as pd

In [20]:
#Lire dans un Dataframe nommé df les données contenues dans le fichier "known.csv"
df=pd.read_csv("known.csv")

In [21]:
#Affichage d'un Aperçu de df
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [22]:
#Verifier si il ya des doublons
print("Quantité de doublons:", df.duplicated().sum())

Quantité de doublons: 0


In [23]:
#Déterminer le taux de valeurs manquantes pour chaque colonne
print(df.isna().mean()*100)

PassengerId     0.000000
HomePlanet      2.312205
CryoSleep       2.496261
Cabin           2.289198
Destination     2.093639
Age             2.059128
VIP             2.335212
RoomService     2.082135
FoodCourt       2.105142
ShoppingMall    2.392730
Spa             2.105142
VRDeck          2.162660
Name            2.300702
Transported     0.000000
dtype: float64


In [24]:
#Afficher le nombre de modalités prises par chaque variable explicative
print("Le nombre de valeurs uniques par colonne:\n")
print(df.nunique())

Le nombre de valeurs uniques par colonne:

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64


## Feature Selection

In [25]:
#Supprimer "Name","PassengerId" et "Cabin"
df.drop(["Name", "PassengerId", "Cabin"], axis=1, inplace=True)

In [26]:
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


## Gestion des valeurs manquantes

In [27]:
#Séparer la variable cible des variables explicatives
from sklearn.model_selection import train_test_split
X= df.drop("Transported",axis=1)
y= df["Transported"]

#Séparer le jeu de données en jeu d'entrainement et un jeu de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)


In [28]:
#Séparer les colonnes numériques et les colonnes catégorielles pour le jeu d’entraînement et le jeu de test
cat_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]
num_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
#Stocker les colonnes numériques dans num_train et num_test et les colonnes catégorielles dans cat_train et cat_test
cat_train, cat_test = X_train[cat_cols], X_test[cat_cols]
num_train, num_test = X_train[num_cols], X_test[num_cols]

In [29]:
#Remplir les valeurs manquantes en utilisant "median" pour les variables numériques et "most_frequent" pour les catégorielles 
from sklearn.impute import SimpleImputer

cat_imputer = SimpleImputer(strategy="most_frequent")
num_imputer = SimpleImputer(strategy="median")

num_train = num_imputer.fit_transform(num_train)
num_test = num_imputer.transform(num_test)

cat_train = cat_imputer.fit_transform(cat_train)
cat_test = cat_imputer.transform(cat_test)


## Encodage des variables catégorielles

In [30]:
#Encodage des variables categorielles avec OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ohe = OneHotEncoder(sparse=False, drop="first")

cat_train = ohe.fit_transform(cat_train)
cat_test = ohe.transform(cat_test)

#Reconstituer les jeux d'entrainement et de test 
X_train = np.concatenate([num_train, cat_train], axis=1)
X_test = np.concatenate([num_test, cat_test], axis=1)

#Affichage des dimentions des jeux reconstitués

print("Dimensions du jeu d’entraînement:", X_train.shape)
print("Dimensions du jeu de test:", X_test.shape)

Dimensions du jeu d’entraînement: (6954, 12)
Dimensions du jeu de test: (1739, 12)


/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Modélisation

## Entraînement 

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

logreg = LogisticRegression()

dt = DecisionTreeClassifier(random_state=42)

# Le scaling des features est optionnel pour l'arbre de décision
scaler = StandardScaler()

num_train_scaled = scaler.fit_transform(num_train)
num_test_scaled = scaler.transform(num_test)

X_train_scaled = np.concatenate([num_train_scaled, cat_train], axis=1)
X_test_scaled = np.concatenate([num_test_scaled, cat_test], axis=1)

# Entraînement de la régression Logistique

logreg.fit(X_train_scaled, y_train)

# Entrainement de l'arbre de décision

dt.fit(X_train, y_train)


DecisionTreeClassifier(random_state=42)

## Évaluation de la performance 

In [33]:
#Comparaison des modèles 
from sklearn.metrics import f1_score

logreg_pred = logreg.predict(X_test_scaled)
print("Score LogReg:", f1_score(y_test, logreg_pred))

dt_pred = dt.predict(X_test)
print("\nScore DecisionTree:", f1_score(y_test, dt_pred))

Score LogReg: 0.7840531561461793

Score DecisionTree: 0.749727965179543


In [34]:
X, y = np.concatenate([X_train, X_test]), np.concatenate([y_train, y_test])

# L'arbre de décision a montré les meilleurs résultats, mais cela peut varier
dt.fit(X, y)

# Chargement des nouvelles données
pred_df = pd.read_csv("missing.csv")

# Feature Engineering

pred_df = pred_df.drop(["Name", "PassengerId", "Cabin"], axis=1)

# Gestion des valeurs manquantes et encodage
cat_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]
num_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]

cat, num = pred_df[cat_cols], pred_df[num_cols]

cat = cat_imputer.transform(cat)
num = num_imputer.transform(num)

cat = ohe.transform(cat)
pred_df = np.concatenate([num, cat], axis=1)

# Prédiction sur les nouvelles données
predictions = dt.predict(pred_df)


In [35]:
print("D'après le modèle, il faudra aller sauver",
      predictions.sum() + y.sum(), "passagers.")


D'après le modèle, il faudra aller sauver 6754 passagers.


## Résolution à l'aide d'une pipeline pour le modèle le plus performant.

In [36]:
# On importe à nouveau df
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
df = pd.read_csv("known.csv")

# On sépare les variables explicatives conservées (X) de la variable cible (y)
X, y = df.drop(["Name", "PassengerId", "Cabin", "Transported"],
               axis=1), df["Transported"]

# On définit les variables catégorielles et les variables numériques
cat_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

num_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]

# On instancie les Transformers qui vont permettre d'imputer les NA
cat_imputer = SimpleImputer(strategy="most_frequent")

num_imputer = SimpleImputer(strategy="median")

# On instancie le Transformer qui va permettre de créer des dummies à partir des catégories
ohe = OneHotEncoder(sparse=False, drop="first")

# On importe les fonctions Pipeline et ColumnTransformer

# On crée une pipeline pour transformer les variables catégorielles
cat_transformers = Pipeline(steps=[('cat_imputer', cat_imputer), ('ohe', ohe)])

# On définit quelles sont les transformations qui vont être appliquées aux différentes variables
columns_transformation = ColumnTransformer(transformers=[(
    "cat", cat_transformers, cat_cols), ("num", num_imputer, num_cols)], remainder="passthrough")

# On crée une pipeline qui rassemble les étapes de préparations et le modèle
pipe = Pipeline(steps=[("col_transform", columns_transformation), ("dt", dt)])

# On entraîne la pipeline
pipe.fit(X, y)

# On importe les données sur lesquelles on effectuera la prédiction
pred_df = pd.read_csv("missing.csv")

# On retire les variables que l'on n'a pas utilisé pour le modèle
pred_df = pred_df.drop(["Name", "PassengerId", "Cabin"], axis=1)

# On fait les prédictions à l'aide de la pipeline
predictions_pipe = pipe.predict(pred_df)

print("D'après le modèle, il faudra aller sauver",
      predictions_pipe.sum() + y.sum(), "passagers.")


D'après le modèle, il faudra aller sauver 6754 passagers.


/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
